In [ ]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops wandb
!pip install scipy

In [1]:
from trl import SFTTrainer
from transformers import TrainingArguments
from peft import LoraConfig, get_peft_model
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer
from datasets import load_dataset

In [2]:
# !pip install pynvml
import pynvml

pynvml.nvmlInit()
handle = pynvml.nvmlDeviceGetHandleByIndex(0)  # GPU index (0 for the first GPU)

# GPU memory usage
memory_info = pynvml.nvmlDeviceGetMemoryInfo(handle)
total_memory = memory_info.total / 1024 / 1024  # Convert to MB
used_memory = memory_info.used / 1024 / 1024  # Convert to MB
free_memory = memory_info.free / 1024 / 1024  # Convert to MB

total_percentage = (total_memory / total_memory) * 100
used_percentage = (used_memory / total_memory) * 100
free_percentage = (free_memory / total_memory) * 100

print("GPU memory total: {} MB".format(total_memory))
print("GPU memory used: {} MB".format(used_memory))
print("GPU memory free: {} MB".format(free_memory))
print("GPU memory total percentage: {:.2f}%".format(total_percentage))
print("GPU memory used percentage: {:.2f}%".format(used_percentage))
print("GPU memory free percentage: {:.2f}%".format(free_percentage))

pynvml.nvmlShutdown()


GPU memory total: 49140.0 MB
GPU memory used: 466.875 MB
GPU memory free: 48673.125 MB
GPU memory total percentage: 100.00%
GPU memory used percentage: 0.95%
GPU memory free percentage: 99.05%


**Load Model/ Download Model**

In [3]:
model_name = "TheBloke/Llama-2-70B-Chat-fp16"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

**Load Tokenizer**

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

**Loading Dataset**

In [6]:
import json
with open("prepared_dataset.json", "r") as f:
    data = list(json.load(f))

print(type(data))


<class 'list'>


In [7]:
from datasets import Dataset

dataset = Dataset.from_list(data)

In [8]:
dataset

Dataset({
    features: ['text'],
    num_rows: 341881
})

**Load Peft config**

In [9]:
lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM"
)

**Loading Trainer**

In [10]:
output_dir = "./results"
per_device_train_batch_size = 1
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 100
logging_steps = 10
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 40000
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
)

**Loading SFTTrainer**

In [11]:
max_seq_length = 512

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

/opt/conda/lib/python3.10/site-packages/peft/utils/other.py:119: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


Map:   0%|          | 0/341881 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:207: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


**Start Training/Fine-tuning**

In [12]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

In [ ]:
trainer.train()

**Saving Model**

In [ ]:
model_to_save = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model  # Take care of distributed/parallel training
model_to_save.save_pretrained("outputs")

**Loading Fine-tuned Model**

In [ ]:
lora_config = LoraConfig.from_pretrained('outputs')
model = get_peft_model(model, lora_config)

**Calling Inference on GPU**

In [ ]:
text = "<Human>: what is blockchain? <Assistant>: "
device = "cuda:0"

inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))